In [1]:
# Import required libraries
import pandas as pd
import numpy as np
from pykalman import KalmanFilter
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

Libraries imported successfully!


In [2]:
# Load the main dataset and outage periods
def load_data():
    """Load the demand data and outage periods"""
    print("Loading Demand_with_Temperature.csv...")
    demand_df = pd.read_csv('Demand_with_Temperature.csv')
    demand_df['DateTime'] = pd.to_datetime(demand_df['DateTime'])
    
    print("Loading tmp/Outage_Runs.csv...")
    outage_df = pd.read_csv('tmp/Outage_Runs.csv')
    outage_df['start'] = pd.to_datetime(outage_df['start'])
    outage_df['end'] = pd.to_datetime(outage_df['end'])
    
    print(f"Demand data shape: {demand_df.shape}")
    print(f"Outage periods: {len(outage_df)} entries")
    print(f"Date range: {demand_df['DateTime'].min()} to {demand_df['DateTime'].max()}")
    
    return demand_df, outage_df

# Load the data
demand_df, outage_df = load_data()

Loading Demand_with_Temperature.csv...
Loading tmp/Outage_Runs.csv...
Demand data shape: (1157172, 17)
Outage periods: 31 entries
Date range: 2013-12-26 01:00:00 to 2024-12-26 00:00:00
Loading tmp/Outage_Runs.csv...
Demand data shape: (1157172, 17)
Outage periods: 31 entries
Date range: 2013-12-26 01:00:00 to 2024-12-26 00:00:00


In [3]:
# Function to identify outlier periods
def identify_outlier_periods(demand_df, outage_df):
    """Mark all data points that fall within outage periods as outliers"""
    demand_df = demand_df.copy()
    demand_df['is_outlier'] = False
    
    print("Identifying outlier periods...")
    for idx, row in outage_df.iterrows():
        start_time = row['start']
        end_time = row['end']
        
        # Mark points within this outage period as outliers
        mask = (demand_df['DateTime'] >= start_time) & (demand_df['DateTime'] <= end_time)
        outlier_count = mask.sum()
        demand_df.loc[mask, 'is_outlier'] = True
        
        print(f"Outage {idx+1}: {start_time} to {end_time} - {outlier_count} points marked as outliers")
    
    total_outliers = demand_df['is_outlier'].sum()
    print(f"\nTotal outlier points identified: {total_outliers} ({total_outliers/len(demand_df)*100:.2f}%)")
    
    return demand_df

# Identify outlier periods
demand_df = identify_outlier_periods(demand_df, outage_df)

Identifying outlier periods...
Outage 1: 2014-03-20 07:05:00 to 2014-03-20 18:00:00 - 132 points marked as outliers
Outage 2: 2014-07-10 10:00:00 to 2014-07-10 16:55:00 - 84 points marked as outliers
Outage 3: 2014-07-16 05:00:00 to 2014-07-18 17:55:00 - 732 points marked as outliers
Outage 4: 2015-02-07 02:05:00 to 2015-02-07 10:00:00 - 96 points marked as outliers
Outage 5: 2015-08-23 01:00:00 to 2015-08-23 07:55:00 - 84 points marked as outliers
Outage 6: 2015-11-23 17:00:00 to 2015-11-23 17:55:00 - 12 points marked as outliers
Outage 7: 2015-11-27 04:00:00 to 2015-11-27 04:55:00 - 12 points marked as outliers
Outage 8: 2016-03-19 11:05:00 to 2016-03-19 17:00:00 - 72 points marked as outliers
Outage 9: 2016-03-30 12:05:00 to 2016-03-30 15:00:00 - 36 points marked as outliers
Outage 10: 2016-12-25 00:00:00 to 2016-12-25 23:55:00 - 288 points marked as outliers
Outage 11: 2016-12-26 12:05:00 to 2016-12-26 14:00:00 - 24 points marked as outliers
Outage 12: 2017-04-20 00:00:00 to 2017-0

In [4]:
# Kalman Filter implementation for smoothing
def apply_kalman_smoothing(demand_series, outlier_mask):
    """Apply Kalman filtering to smooth the demand data and replace outliers"""
    print("Applying Kalman filtering...")
    
    # Prepare the data - use observations where we don't have outliers
    observations = demand_series.values.copy()
    
    # Create a masked array where outliers are marked as missing
    masked_observations = np.ma.masked_array(observations, mask=outlier_mask)
    
    # Set up Kalman Filter
    # State transition matrix (assumes trend continues)
    transition_matrix = np.array([[1, 1], [0, 1]])
    
    # Observation matrix (we observe the level)
    observation_matrix = np.array([[1, 0]])
    
    # Initialize Kalman Filter
    kf = KalmanFilter(
        transition_matrices=transition_matrix,
        observation_matrices=observation_matrix,
        initial_state_mean=[observations[0], 0],  # Start with first observation and zero trend
        n_dim_state=2
    )
    
    # Fit the model using EM algorithm on non-outlier data
    print("Fitting Kalman Filter using EM algorithm...")
    kf = kf.em(masked_observations, n_iter=50)
    
    # Apply smoothing to get the best estimates
    print("Applying Kalman smoothing...")
    state_means, state_covariances = kf.smooth(masked_observations)
    
    # Extract the smoothed values (level component)
    smoothed_values = state_means[:, 0]
    
    return smoothed_values

# Apply Kalman smoothing
smoothed_demand = apply_kalman_smoothing(demand_df['TOTAL'], demand_df['is_outlier'])

Applying Kalman filtering...
Fitting Kalman Filter using EM algorithm...


KeyboardInterrupt: 

In [ ]:
# Create the cleaned dataset
def create_cleaned_dataset(demand_df, smoothed_demand):
    """Create the final cleaned dataset with outliers replaced by Kalman smoothed values"""
    cleaned_df = demand_df.copy()
    
    # Replace outlier values with smoothed values
    cleaned_df['TOTAL_original'] = cleaned_df['TOTAL'].copy()
    cleaned_df.loc[cleaned_df['is_outlier'], 'TOTAL'] = smoothed_demand[cleaned_df['is_outlier']]
    
    # Add a column to track which values were smoothed
    cleaned_df['smoothed'] = cleaned_df['is_outlier'].copy()
    
    # Calculate statistics
    outlier_count = cleaned_df['is_outlier'].sum()
    print(f"Replaced {outlier_count} outlier values with Kalman smoothed estimates")
    
    return cleaned_df

# Create cleaned dataset
cleaned_df = create_cleaned_dataset(demand_df, smoothed_demand)

# Display some statistics
print("\nCleaning Statistics:")
print(f"Total data points: {len(cleaned_df)}")
print(f"Outlier points replaced: {cleaned_df['is_outlier'].sum()}")
print(f"Percentage of data smoothed: {cleaned_df['is_outlier'].sum()/len(cleaned_df)*100:.2f}%")

# Show sample of cleaned data
print("\nSample of cleaned data:")
print(cleaned_df[['DateTime', 'TOTAL_original', 'TOTAL', 'smoothed', 'Temperature']].head(10))

In [ ]:
# Visualization to show the effect of outlier removal
def plot_comparison(cleaned_df, sample_start='2014-03-19', sample_end='2014-03-21'):
    """Plot comparison between original and cleaned data for a sample period"""
    
    # Filter for sample period around one of the outages
    sample_mask = (cleaned_df['DateTime'] >= sample_start) & (cleaned_df['DateTime'] <= sample_end)
    sample_df = cleaned_df[sample_mask].copy()
    
    if len(sample_df) == 0:
        print(f"No data found for period {sample_start} to {sample_end}")
        return
    
    plt.figure(figsize=(15, 8))
    
    # Plot original data
    plt.subplot(2, 1, 1)
    plt.plot(sample_df['DateTime'], sample_df['TOTAL_original'], 'b-', label='Original Data', alpha=0.7)
    outlier_mask = sample_df['is_outlier']
    if outlier_mask.any():
        plt.scatter(sample_df[outlier_mask]['DateTime'], 
                   sample_df[outlier_mask]['TOTAL_original'], 
                   color='red', s=20, label='Outliers', zorder=5)
    plt.title('Original Data with Outliers Highlighted')
    plt.ylabel('Demand (TOTAL)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Plot cleaned data
    plt.subplot(2, 1, 2)
    plt.plot(sample_df['DateTime'], sample_df['TOTAL'], 'g-', label='Cleaned Data (Kalman Smoothed)', linewidth=2)
    if outlier_mask.any():
        plt.scatter(sample_df[outlier_mask]['DateTime'], 
                   sample_df[outlier_mask]['TOTAL'], 
                   color='orange', s=20, label='Smoothed Values', zorder=5)
    plt.title('Cleaned Data with Kalman Smoothed Values')
    plt.ylabel('Demand (TOTAL)')
    plt.xlabel('DateTime')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Sample period: {sample_start} to {sample_end}")
    print(f"Data points in sample: {len(sample_df)}")
    print(f"Outlier points in sample: {outlier_mask.sum()}")

# Create visualization
plot_comparison(cleaned_df)

In [ ]:
# Save the cleaned dataset
def save_cleaned_data(cleaned_df, filename='Demand_smoothed_kalman.csv'):
    """Save the cleaned dataset to CSV file with same structure as original"""
    
    # Get original column order from the input dataset
    original_columns = ['DateTime','TOTAL','Outage','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday','Holiday','ECQ','GCQ','ALERT LEVEL 1','ALERT LEVEL 2','ALERT LEVEL 3','Temperature']
    
    # Prepare final dataset - keep only original columns with cleaned TOTAL values
    final_df = cleaned_df[original_columns].copy()
    
    # Save to CSV with exact same structure as original
    final_df.to_csv(filename, index=False)
    print(f"Cleaned dataset saved to: {filename}")
    print(f"Shape: {final_df.shape}")
    print(f"Columns: {list(final_df.columns)}")
    print(f"Structure matches original Demand_with_Temperature.csv")
    
    # Show how many values were actually replaced
    outlier_count = cleaned_df['is_outlier'].sum()
    print(f"Values replaced with Kalman smoothing: {outlier_count}")
    
    return final_df

# Save the cleaned dataset
final_cleaned_df = save_cleaned_data(cleaned_df)

print("\nFirst few rows of saved dataset:")
print(final_cleaned_df.head())

In [ ]:
# Summary and validation
def generate_summary(original_df, cleaned_df, outage_df):
    """Generate summary statistics about the outlier removal process"""
    
    print("=== OUTLIER REMOVAL SUMMARY ===")
    print(f"Original dataset shape: {original_df.shape}")
    print(f"Cleaned dataset shape: {cleaned_df.shape}")
    print(f"Number of outage periods processed: {len(outage_df)}")
    
    # Calculate outlier statistics
    total_outliers = cleaned_df['smoothed'].sum() if 'smoothed' in cleaned_df.columns else 0
    print(f"\nOutlier Statistics:")
    print(f"Total outlier points identified: {total_outliers}")
    print(f"Percentage of data smoothed: {total_outliers/len(cleaned_df)*100:.2f}%")
    
    # Show range of smoothed values vs original
    if 'TOTAL_original' in cleaned_df.columns:
        smoothed_mask = cleaned_df['smoothed'] == True
        if smoothed_mask.any():
            original_outlier_values = cleaned_df[smoothed_mask]['TOTAL_original']
            smoothed_values = cleaned_df[smoothed_mask]['TOTAL']
            
            print(f"\nValue Comparison for Smoothed Points:")
            print(f"Original outlier values - Min: {original_outlier_values.min():.2f}, Max: {original_outlier_values.max():.2f}, Mean: {original_outlier_values.mean():.2f}")
            print(f"Smoothed values - Min: {smoothed_values.min():.2f}, Max: {smoothed_values.max():.2f}, Mean: {smoothed_values.mean():.2f}")
    
    # Show some examples of outage periods that were processed
    print(f"\nOutage Periods Processed:")
    for idx, row in outage_df.head().iterrows():
        duration = row['duration_hours']
        print(f"  {idx+1}. {row['start']} to {row['end']} ({duration:.1f} hours)")
    
    if len(outage_df) > 5:
        print(f"  ... and {len(outage_df)-5} more periods")
    
    print(f"\nCleaned dataset saved as: 'Demand_smoothed_kalman.csv'")
    print("Process completed successfully!")

# Generate final summary
generate_summary(demand_df, final_cleaned_df, outage_df)